# closure phases

[L. Blackburn, Mar 2018]

The purpose of this test is to check trivial and non-trivial closure phases for sanity.

In [ ]:
# basic import and helper functions
import pandas as pd
from eat.io import hops, util
from eat.hops import util as hu
from eat.plots import util as pu
from collections import OrderedDict
import itertools
import matplotlib.pyplot as plt
import numpy as np
import os
from matplotlib.ticker import MultipleLocator
import seaborn as sns
# sns.set_palette(sns.color_palette(sns.hls_palette(16, l=.6, s=.6)))
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
# %config InlineBackend.figure_formats=['svg']
import sys
nb_stdout = sys.stdout # grab for later
def wide(w=8, h=3): plt.setp(plt.gcf(), figwidth=w, figheight=h); \
    plt.tight_layout()
def tightx(): plt.autoscale(enable=True, axis='x', tight=True)
def multline(xs, fun=plt.axvline):
    for x in xs: fun(x, alpha=0.25, ls='--', color='k')
def toiter(x):
    return(x if hasattr(x, '__iter__') else [x,])
# pd.options.display.float_format = '{:,.6f}'.format
from IPython.core.display import display, HTML
display(HTML("<style>"
    + "#notebook { padding-top:0px !important; } " 
    + ".container { width:100% !important; } "
    + ".end_space { min-height:0px !important; } "
    + "</style>"))
# Run this in Python once, it should take effect permanently
from notebook.services.config import ConfigManager
c = ConfigManager()
_ = c.update('notebook', {"CodeCell": {"cm_config": {"autoCloseBrackets": False}}})
import string

In [ ]:
os.getcwd()

In [ ]:
reload(util)
# define and load data
# stage = '3.+adhoc'
# datadir = '/data/2017-april/ce/er3/hops-lo/%s/data/' % stage
datadir = '../' + os.environ['DATADIR'] + '/'
a = hops.read_alist(datadir + 'alist.v6') # alist file to make scan_no
ll = hops.read_tlist_v6(datadir + 'alist.v6.8s.LL.close.avg')
ll['polarization'] = 'LL'
rr = hops.read_tlist_v6(datadir + 'alist.v6.8s.RR.close.avg')
rr['polarization'] = 'RR'
c = pd.concat((ll, rr), ignore_index=True)
util.add_gmst(c)
hu.setparity(c)

In [ ]:
util.add_scanno(a)
tup2scanno = a.groupby(['expt_no', 'scan_id']).first().scan_no
c = c.join(tup2scanno, on=['expt_no', 'scan_id'], how='left')

In [ ]:
# expt_no boundary ticker marks vs scan
elines = a.scan_no.sort_values().values[np.nonzero(np.diff(a.expt_no.sort_values()) > 0)[0]] + 0.5

In [ ]:
def clplot(c, triangles, threshold=3):
    d = c[(c.bis_snr > threshold) & ~c.triangle.str.contains('R') & (c.duration > 8*5)]
    for tri in triangles:
        rr = d[(d.polarization == 'LL') & (d.triangle==tri)]
        ll = d[(d.polarization == 'RR') & (d.triangle==tri)]
        hl = plt.errorbar(ll.scan_no - 0.025, ll.bis_phas, yerr=(1./ll.bis_snr)*180./np.pi,
                          fmt='o', label=tri)
        hr = plt.errorbar(rr.scan_no + 0.025, rr.bis_phas, yerr=(1./rr.bis_snr)*180./np.pi,
                          fmt='x', label='_nolabel_', color=hl[0].get_color())
    plt.gca().yaxis.grid(alpha=0.25)
    plt.legend(ncol=3)
    plt.title('closure phases')
    plt.xlabel('scan number')
    plt.ylabel('degrees')
    pu.multline(elines)

In [ ]:
clplot(c, sorted((t for t in set(c.triangle) if 'A' in t and 'X' in t and 'R' not in t)))
wide(12, 5)

In [ ]:
clplot(c, sorted((t for t in set(c.triangle) if 'S' in t and 'J' in t and 'R' not in t)))
wide(12, 5)

In [ ]:
def clplot2(c, source, triangles, threshold=3):
    d = c[(c.bis_snr > threshold) & ~c.triangle.str.contains('R') & (c.duration > 8*5) & (c.source==source)]
    for tri in triangles:
        rr = d[(d.polarization == 'LL') & (d.triangle==tri)]
        ll = d[(d.polarization == 'RR') & (d.triangle==tri)]
        hl = plt.errorbar(ll.gmst - 0.025, ll.bis_phas, yerr=(1./ll.bis_snr)*180./np.pi,
                          fmt='o', label=tri)
        hr = plt.errorbar(rr.gmst + 0.025, rr.bis_phas, yerr=(1./rr.bis_snr)*180./np.pi,
                          fmt='x', label='_nolabel_', color=hl[0].get_color())
    plt.gca().yaxis.grid(alpha=0.25)
    plt.legend(ncol=3)
    plt.title(source + ' closure phases')
    plt.xlabel('GMST hour')
    plt.ylabel('degrees')

In [ ]:
triangles = sorted((t for t in set(c.triangle) if 'A' in t and 'X' not in t and 'R' not in t))
triangles = [t for t in triangles if not ('J' in t and 'S' in t)]
import pandas as pd
import seaborn as sns
sns.set_palette(sns.color_palette(sns.hls_palette(len(triangles), l=.6, s=.6)))
for src in sorted(set(c.source)):
    clplot2(c, source=src, triangles=triangles)
    plt.ylim(-180, 180)
    wide(12, 5)
    plt.show()